## Setup Environment

In [ ]:
import pandas as pd
import ecoscope

ecoscope.init(selenium=True)

Executing: /tmp/apt-key-gpghome.d9NPzveAqt/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.5vLrKqabYt/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.1BtmJIErzn/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 https://cloud.r-project.org/bin/l

In [ ]:
ER_PASSWORD = ""

## Parameters

In [ ]:
# EarthRanger Credentials
ER_SERVER = "https://mep-dev.pamdas.org"
ER_USERNAME = "yun.wu"
#ER_PASSWORD = Variable.get('er_password')
#ER_PASSWORD = os.getenv("er_password", "")

# SubjectGroup
subjectgroup_name = "Elephants"

# Analysis Time Range
start = "2000-01-01"
end = "2023-07-01"

# Relocations Bounding Box Filter Coordinates
relocs_coord_filter_min_x = -5.0
relocs_coord_filter_max_x = 1.0
relocs_coord_filter_min_y = 12.0
relocs_coord_filter_max_y = 18.0

# Relocations Filter Point Coordinates
relocs_filter_point_coords = [[180, 90], [0, 0]]

# Trajectory Filter Settings
traj_min_length_meters=0.0
traj_max_length_meters=10000
traj_min_time_secs=120.0
traj_max_time_secs=4 * 60 * 60
traj_min_speed_kmhr=0.000001
traj_max_speed_kmhr=8.0

Mounted at /content/drive/


## Logic

### Connect to EarthRanger

In [ ]:
er_io = ecoscope.io.EarthRangerIO(
    server=ER_SERVER,
    username=ER_USERNAME,
    password=ER_PASSWORD,
    tcp_limit=5,
    sub_page_size=4000,
)

Please enter your ER password: ··········


### Relocations

In [ ]:
%%time
relocs = er_io.get_subjectgroup_observations(
    group_name=subjectgroup_name,
    include_inactive=True,
    include_subject_details=True,
    include_source_details=False,
    include_subjectsource_details=False,
    filter=0,
    since=pd.to_datetime(start).isoformat(),
    until=pd.to_datetime(end).isoformat(),
    include_details=True,
)

  0%|          | 0/2 [00:00<?, ?it/s]

CPU times: user 375 ms, sys: 85.1 ms, total: 460 ms
Wall time: 4.08 s


In [ ]:
relocs = relocs[relocs.extra__subject__name != 'Habiba'] # temporary hack until relocs_filter method is fixed...

In [ ]:
# coord_filter = ecoscope.base.RelocsCoordinateFilter(
#     min_x=relocs_coord_filter_min_x,
#     max_x=relocs_coord_filter_max_x,
#     min_y=relocs_coord_filter_min_y,
#     max_y=relocs_coord_filter_max_x,
#     filter_point_coords=relocs_filter_point_coords,
# )
# relocs.apply_reloc_filter(coord_filter, inplace=True)
# relocs.remove_filtered(inplace=True)

In [ ]:
%%time
## Clean up columns that are not compatible with a GPKG format
relocs.drop(
    columns=relocs.columns[relocs.applymap(lambda x: isinstance(x, list)).any()],
    errors="ignore",
    inplace=True,
)

CPU times: user 63.8 ms, sys: 0 ns, total: 63.8 ms
Wall time: 67.4 ms


## Trajectory

In [ ]:
%%time
## create a trajectory
traj = ecoscope.base.Trajectory.from_relocations(relocs)

CPU times: user 180 ms, sys: 261 µs, total: 180 ms
Wall time: 450 ms


In [ ]:
## filter the trajectory
traj_seg_filter = ecoscope.base.TrajSegFilter(
    min_length_meters=traj_min_length_meters,
    max_length_meters=traj_max_length_meters,
    min_time_secs=traj_min_time_secs,
    max_time_secs=traj_max_time_secs,
    min_speed_kmhr=traj_min_speed_kmhr,
    max_speed_kmhr=traj_max_speed_kmhr,
)
traj.apply_traj_filter(traj_seg_filter, inplace=True)
traj.remove_filtered(inplace=True)

In [ ]:
%%time
## remove any incompatible columns for geopackage export
traj.drop(
    columns=traj.columns[traj.applymap(lambda x: isinstance(x, list)).any()],
    errors="ignore",
    inplace=True,
)

CPU times: user 57.1 ms, sys: 0 ns, total: 57.1 ms
Wall time: 58.3 ms


## Outputs

In [ ]:
# Calculate the number of rows in the geodatabse
len(relocs)

4500

In [ ]:
len(traj)

2679

In [ ]:
# Map of the data
m = ecoscope.mapping.EcoMap(static=True)
m.add_tile_layer(
    url="https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}",
    name="Google Satellite",
    attribution="Google",
    opacity=1.0,
)
m.add_speedmap(trajectory=traj, classification_method="equal_interval", num_classes=6, bins=None)
m.add_north_arrow(position="topright")
m.add_title(title="Elephant Speed Map", align="center", font_size="18px")
m.zoom_to_gdf(traj)
m

In [ ]:
# Export to HTML
source_file_name = "home/airflow/gcs/data/ecomap.html"
m.to_html(source_file_name)

In [ ]:
# Upload to Google Cloud Storage
from google.cloud import storage

storage_client = storage.Client()
bucket = storage_client.get_bucket("us-central1-mep-airflow-env-f3a7f84d-bucket")
blob = bucket.blob("website/ecomap.html")

blob.upload_from_filename(source_file_name)

print('File {} uploaded to {}.'.format(
    source_file_name,
    "website"))